# Build Chatbot
`11_chatbot.ipynb`
- https://python.langchain.com/docs/tutorials/chatbot/

In [ ]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv()

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage

messages = [
    HumanMessage(content='Hi!, I am bob.'),
    AIMessage(content='Hello bob. how can I help you.'),
    HumanMessage(content='Say my name.')
]

llm = ChatOpenAI(model='gpt-4.1-nano', temperature=0)

res = llm.invoke(messages)

res.pretty_print()

In [ ]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, END, MessagesState, StateGraph

# Graph Builider
builder = StateGraph(state_schema=MessagesState)

# Node
def simple_node(state: MessagesState):
    res = llm.invoke(state['messages'])
    return {'messages': res}

builder.add_node('simple_node', simple_node)

# Edge (Node 끼리 연결)
builder.add_edge(START, 'simple_node')
builder.add_edge('simple_node', END)

# Memory (대화내역 기록)
memory = MemorySaver()

# Graph (그래프 생성)
graph = builder.compile(checkpointer=memory)

In [ ]:
# 설정(conf, config, configuration -> 설정)
config = {'configurable': {'thread_id': 'abc123'}}  # 채팅방 아이디 (바뀌면 다른 대화가 된다.)

graph.invoke({'messages': messages}, config=config)

In [ ]:
import uuid

u_id = uuid.uuid1()
print(u_id)

config = {'configurable': {'thread_id': '가나다123'}}  # 채팅방 아이디 -> 추후에는 UUID 형식으로 생성
messages = [
    HumanMessage(content='say my name.')
]
graph.invoke({'messages': messages}, config=config)

## Langgraph + `PromptTemplate`

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages([
    ('system', '너는 유능한 어시스턴트야. 너의 능력을 최대한 활용해서 답을 해봐.'),
    MessagesPlaceholder(variable_name='messages')  # 모든 저장된 대화 내용(최신것 포함)
])

# 실행 예시
for msg in prompt_template.invoke({'messages': ['hi']}).messages:
    print(msg)

## State 확장

In [ ]:
# 내장된 MessagesState를 확장해서 사용
class MyState(MessagesState):
    # 상속받아서 이미 key 'messages'는 있음
    # messages: Annotated[list[AnyMessage], add_messages]
    lang: str

builder = StateGraph(state_schema=MyState)

prompt_template = ChatPromptTemplate.from_messages([
    ('system', '너는 유능한 어시스턴트야. 너의 능력을 최대한 활용해서 답을 해봐. {lang} 언어로 답해.'),
    MessagesPlaceholder(variable_name='messages')  # 모든 저장된 대화 내용(최신것 포함)
])

def simple_node(state: MyState):
    # prompt 추가.
    chain = prompt_template | llm  # 체인 방식
    res = chain.invoke(state)

    return {'messages': res}

builder.add_node('simple_node', simple_node)

builder.add_edge(START, 'simple_node')
builder.add_edge('simple_node', END)

memory = MemorySaver()

graph = builder.compile(checkpointer=memory)

In [ ]:
config = {'configurable': {'thread_id': 'abc1'}} 
state = {
    'messages': [HumanMessage(content='나는 서울에 있어')],
    'lang': 'Spanish'
}

res = graph.invoke(state, config)

for msg in res['messages']:
    msg.pretty_print()

## 대화 기록 관리하기
대확 내역을 관리하지 않으면, LLM의 컨텍스트 윈도우(입력 최대치)를 넘어가 버림.


In [ ]:
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage, trim_messages

# trim - 정리하다
trimmer = trim_messages(
    strategy='last',      # 최신 메세지들을
    max_tokens=65,         # 최대 65토큰 까지만 허용,
    token_counter=llm,    # llm 모델에 맞춰서 토큰 세고
    include_system=True,  # system 프롬프트는 포함(정리X)
    allow_partial=False,  # 메세지 중간에서 자르지는 말고
    start_on='human',     # 잘린 메세지의 첫번째는 사람 메세지가 되도록
    
)

messages = [
    SystemMessage(content="you're a good assistant"),

    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)  # 시스템메시지 -> 2+2 부터 등장

In [76]:
class MyState(MessagesState):
    # messages: ~~
    lang: str


builder = StateGraph(state_schema=MyState)

prompt_template = ChatPromptTemplate.from_messages([
    ('system', '너는 유능한 어시스턴트야. 너의 능력을 최대한 활용해서 답을 해봐. {lang} 언어로 답해.'),
    MessagesPlaceholder(variable_name='messages')  # 모든 저장된 대화 내용(최신것 포함)
])


def simple_node(state: MyState):
    # 메세지 정리 -> 프롬프트 생성 -> LLM 답변
    # print('정리 전 메시지 개수: ', len(state['messages']))
    trimmed_messages = trimmer.invoke(state['messages'])
    # print('정리 후 메시지 개수: ', len(trimmed_messages))
    
    # 체인 생성
    chain = prompt_template | llm
    
    # 정리된 메세지로 state 교체 후, 체인 실행
    state['messages'] = trimmed_messages
    res = chain.invoke(state)

    return {'messages': [res]}


builder.add_node('simple_node', simple_node)

builder.add_edge(START, 'simple_node')
builder.add_edge('simple_node', END)

memory = MemorySaver()

graph = builder.compile(checkpointer=memory)

In [81]:
config = {'configurable': {'thread_id': 'abc1'}} 
state = {
    'messages': [HumanMessage(content='내 상태는 어때')],
    'lang': '한국어'
}

for chunk, metadata in graph.stream(state, config, stream_mode='messages'):
    print(chunk.content, end='|')

|현재| 당신|의| 상태|에| 대해| 구|체|적인| 정보|가| 없|어서| 정확|히| 말씀|드|리|기| 어렵|습니다|.| 몸| 상태|,| 감|정| 상태|,| 또는| 다른| 어떤| 면|에| 대해| 말씀|해|주|시면| 더| 도움|을| 드|릴| 수| 있을| 것| 같|아요|.| 어떤| 부분|이| 걱|정|되|거나| 궁|금|하|신|지| 알려|주세요|.||